In [ ]:
! pip install datetime pandas matplotlib

In [ ]:
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
user_input = '/Users/atharvchandratre/Workspace/Atharv/GitHub/PrismBenchmarkAnalyzer/prism_n3_le20_d202_vchain_1.log' # Enter path to log here
assert os.path.exists(user_input), "I did not find the file at "+str(user_input)
f = open(user_input,'r')
print("Log File found, beginning analysis...")
f.close()

In [ ]:
ProposerBlockMined = 'ProposerBlockMined'
ValidProposerBlockDownloaded = 'ValidProposerBlockDownloaded'
ProposerBlockConfirmed = 'ProposerBlockConfirmed'
ProposerBlockRolledBack = 'ProposerBlockRolledBack'
epoch = datetime.utcfromtimestamp(0)

hashToNumber = {}
hashToCreationTimestamp = {}
hashToCreationEvent = {}
hashToConfirmationTimestamp = {}
hashToRollbackTimestamp = {}

In [ ]:
def getTimestamp(timestr):
    dateTimeObj = datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S,%f')
    delta = dateTimeObj - epoch
    return delta.total_seconds()*1000

In [ ]:
with open(user_input, 'r') as f:
    for line in f:
        if "BENCHMARKING:" not in line:
            continue
        benchmarkSubstrings = line[line.find("BENCHMARKING"):-1].split(':')
        dateTimeString = line.split('[')[0][:-1]
        timestamp = getTimestamp(dateTimeString)
        event = benchmarkSubstrings[1]
        blockNumber = benchmarkSubstrings[3]
        blockHash = benchmarkSubstrings[-1][1:-1]
        if event == ProposerBlockMined or event == ValidProposerBlockDownloaded:
            if hashToCreationTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToCreationEvent[blockHash] = event
            hashToCreationTimestamp[blockHash] = timestamp
        if event == ProposerBlockConfirmed:
            if hashToConfirmationTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToConfirmationTimestamp[blockHash] = timestamp
        if event == ProposerBlockRolledBack:
            if hashToRollbackTimestamp.get(blockHash)!=None:
                continue
            hashToNumber[blockHash] = blockNumber
            hashToRollbackTimestamp[blockHash] = timestamp

f.close()

In [ ]:
df = pd.DataFrame(columns=['Block Hash','Block Number','Timestamp - Creation','Event','Timestamp - Confirmation','Timestamp - Rollback'])

for index,key in enumerate(hashToNumber.keys()):
    df.loc[index] = [key,
        int(hashToNumber.get(key,'')),
        int(hashToCreationTimestamp.get(key,-1)),
        hashToCreationEvent.get(key,''),
        int(hashToConfirmationTimestamp.get(key,-1)),
        int(hashToRollbackTimestamp.get(key,-1))]

df.sort_values('Block Number')

In [ ]:
df['Latency'] = df[df['Timestamp - Confirmation']!=-1]['Timestamp - Confirmation']-df[df['Timestamp - Confirmation']!=-1]['Timestamp - Creation']
df['Mining Time'] = df['Timestamp - Creation'].diff()
df['Confirmation Depth']=df['Latency']/df['Mining Time']

In [ ]:
print("Mean Latency =",df['Latency'].dropna().mean())
print("Median Latency =",df['Latency'].dropna().median())
print("Mean Mining Time =",df['Mining Time'].dropna().mean())
print("Median Mining Time =",df['Mining Time'].dropna().median())
print("Mean Confirmation Depth =",df['Confirmation Depth'].dropna().mean())
print("Median Confirmation Depth =",df['Confirmation Depth'].dropna().median())

In [ ]:
creationTimestamps = df['Timestamp - Creation']
confirmationTimestamps = df[df['Timestamp - Confirmation']!=-1]['Timestamp - Confirmation']
insertionPoints = pd.DataFrame(creationTimestamps.searchsorted(value=confirmationTimestamps),columns=['Insertion Points'])
cdArray = insertionPoints['Insertion Points'] - insertionPoints['Insertion Points'].index
print("Trial Mean Confirmation Depth =",cdArray.mean())
print("Trial Median Confirmation Depth =",cdArray.median())

In [ ]:
proposerBlockNumbersArray = []
voterChainBestTopBlocksArray = []
with open(user_input, 'r') as f:
    for line in f:
        if "VOTERCHAINHEALTH:" not in line:
            continue
        benchmarkSubstrings = line[line.find("VOTERCHAINHEALTH"):-1].split(':')
        proposerBlockNumber = int(benchmarkSubstrings[-2][:(benchmarkSubstrings[-2].find("(")-1)])
        voterChainBestTopBlock = int(benchmarkSubstrings[4])
        proposerBlockNumbersArray.append(proposerBlockNumber)
        voterChainBestTopBlocksArray.append(voterChainBestTopBlock)


f.close()

In [ ]:
plt.scatter(proposerBlockNumbersArray, voterChainBestTopBlocksArray, c ="orange")
 
# To show the plot
plt.xlabel("Proposer Block Number")
plt.ylabel("Voter Chain Best Top Block")
plt.show()

In [ ]:
firstTimeStamp = df['Timestamp - Creation'][0]

proposerBlocksCoordinates = [[],[]]
proposerBlocksMinedOnlyCoordinates = [[],[]]
proposerBlockDownloadedOnlyCoordinates = [[],[]]

with open(user_input, 'r') as f:
    for line in f:
        if "BENCHMARKING:" not in line:
            continue
        benchmarkSubstrings = line[line.find("BENCHMARKING"):-1].split(':')
        dateTimeString = line.split('[')[0][:-1]
        timestamp = getTimestamp(dateTimeString)
        event = benchmarkSubstrings[1]
        blockNumber = int(benchmarkSubstrings[3])
        if event == ProposerBlockMined:
            proposerBlocksCoordinates[0].append(blockNumber)
            proposerBlocksCoordinates[1].append((timestamp-firstTimeStamp)/1000)
            proposerBlocksMinedOnlyCoordinates[0].append(blockNumber)
            proposerBlocksMinedOnlyCoordinates[1].append((timestamp-firstTimeStamp)/1000)
        if event == ValidProposerBlockDownloaded:
            proposerBlocksCoordinates[0].append(blockNumber)
            proposerBlocksCoordinates[1].append((timestamp-firstTimeStamp)/1000)
            proposerBlockDownloadedOnlyCoordinates[0].append(blockNumber)
            proposerBlockDownloadedOnlyCoordinates[1].append((timestamp-firstTimeStamp)/1000)
f.close()


voterChainsCoordinates = []
for i in range(0,100):
    voterChainsCoordinates.append([[],[]])

with open(user_input, 'r') as f:
    for line in f:
        if "VOTERCHAINHEALTH:" not in line:
            continue
        benchmarkSubstrings = line[line.find("VOTERCHAINHEALTH"):-1].split(':')
        dateTimeString = line.split('[')[0][:-1]
        timestamp = getTimestamp(dateTimeString)
        voterChainBestTopBlock = int(benchmarkSubstrings[4])
        voterChainIdx = int(benchmarkSubstrings[2])
        voterChainsCoordinates[voterChainIdx][0].append(voterChainBestTopBlock)
        voterChainsCoordinates[voterChainIdx][1].append((timestamp-firstTimeStamp)/1000)
f.close()

In [ ]:
voterChainToSee = 80

plt.plot(proposerBlocksCoordinates[1], proposerBlocksCoordinates[0], c ="forestgreen", label="Proposer Chain")
plt.plot(voterChainsCoordinates[voterChainToSee][1], voterChainsCoordinates[voterChainToSee][0], c="red", label="Voter Chain") 
plt.legend(loc="upper left")

 
# To show the plot
plt.title("One Proposer Chain with One Voter Chain")
plt.xlabel("Timestamp (seconds)")
plt.ylabel("Block Number")
plt.show()

In [ ]:
numberOfChainsToSee = 20

for i in range(0,numberOfChainsToSee):
    plt.plot(voterChainsCoordinates[i][1], voterChainsCoordinates[i][0], c="lightgreen", label="Voter Chain")

plt.plot(proposerBlocksCoordinates[1], proposerBlocksCoordinates[0], c ="red", label="Proposer Chain") 

# To show the plot
plt.title("One Proposer Chain with Multiple Voter Chains")
plt.xlabel("Timestamp (seconds)")
plt.ylabel("Block Number")
plt.show()

In [ ]:
voterChainToSee = 42

plt.plot(proposerBlocksMinedOnlyCoordinates[1], proposerBlocksMinedOnlyCoordinates[0], c ="forestgreen", label="Proposer Chain")
plt.plot(voterChainsCoordinates[voterChainToSee][1], voterChainsCoordinates[voterChainToSee][0], c="red", label="Voter Chain") 
plt.legend(loc="upper left")

 
# To show the plot
plt.title("Proposer Chain Mined Only with One Voter Chain")
plt.xlabel("Timestamp (seconds)")
plt.ylabel("Block Number")
plt.show()

In [ ]:
voterChainToSee = 42

plt.plot(proposerBlockDownloadedOnlyCoordinates[1], proposerBlockDownloadedOnlyCoordinates[0], c ="forestgreen", label="Proposer Chain")
plt.plot(voterChainsCoordinates[voterChainToSee][1], voterChainsCoordinates[voterChainToSee][0], c="red", label="Voter Chain") 
plt.legend(loc="upper left")

 
# To show the plot
plt.title("Proposer Chain Downloaded Only with One Voter Chain")
plt.xlabel("Timestamp (seconds)")
plt.ylabel("Block Number")
plt.show()

In [ ]:
plt.scatter(proposerBlockNumbersArray, voterChainBestTopBlocksArray, c ="orange")
 
# To show the plot
plt.title("Proposer Block Number vs Voter Chains Best Top Block Scatter")
plt.xlabel("Proposer Block Number")
plt.ylabel("Voter Chain Best Top Block")
plt.show()